# GIN-forkに公開されているリポジトリから実験の入力データを用意する

[GIN-fork](http://dg02.dg.rcos.nii.ac.jp/)に公開されているあなたのリポジトリや他の研究者のリポジトリのデータをこの実験の入力データとして用意します。  

指定したリポジトリ内のファイルの実データを一括でダウンロードすることも、実験に必要なファイルの実データのみをインストールすることも可能です。  
ただし、多くのデータおよび大容量のデータをダウンロードする場合は、時間がかかる可能性があります。一括でダウンロードするよりも、必要なファイルのみをダウンロードすることでダウンロード時間の減少が見込めます。

上から順番にセルを実行してください。

※もし入力データとして利用したいファイルの実データが「git-annex」によってAWS S3準拠のオブジェクトストレージに保存されている場合、[こちら](util/operate_s3_annex.ipynb)を実行することで当該データを取得できます。

以下の図は、このタスクを行う際のユースケースです。ご参照ください。

<img src="./../../../files/WORKFLOWS/EX-WORKFLOWS/images/gin-forkユースケース.png" width=80%>

In [ ]:
!pip install panel

## 1. データのあるリポジトリのURLおよび実験名を入力する

以下のセルを実行して、GIN-fork上の入力データにしたいデータのある研究リポジトリのGIT URLおよび実験名を入力してください。  
GIT URLについては、以下の図を参考に、GIN-forkの研究リポジトリの画面右上でコピーしてください。

<img src="./../../../files/WORKFLOWS/EX-WORKFLOWS/images/gitURL.png" width=60%>

入力したURLの履歴をこのノートブックに保存したい場合は、実行後にセルの左側に表示されるピンマークをクリックしてください。  
※入力に誤りがある場合は、再度このセルを実行して下さい。

In [ ]:
from ipywidgets import Text, Button, Layout
from IPython.display import clear_output

def on_click_callback(clicked_button: Button) -> None:
    global input_repo
    global input_repo_title
    global input_repo_path
    input_repo = text.value
    input_repo_title = input_repo.split('/')[-1].replace('.git', '')
    input_repo_path = '/home/jovyan/.tmp/' + input_repo_title
    clear_output()
    print("入力完了しました：", input_repo)

# テキストボックス
text = Text(
    description='URL：',
    placeholder='http://dg02.dg.rcos.nii.ac.jp/user/repository_title.git',
    layout=Layout(width='500px')
)
button = Button(description='入力完了')
button.on_click(on_click_callback)
text.on_submit(on_click_callback)
display(text, button)

## 2. 必要なデータを持つ実験名を選択する

1.で入力したリポジトリの実験群から、必要なデータを持つ実験名を選択してください。  
パラメータ実験の場合は、パラメータ実験名も選択してください。  
複数の実験からファイルを用意したい場合は、各実験毎に手順2.～?を繰り返し実行してください。

In [ ]:
# 以下で入力リポジトリの選択GUIを表示するために軽量にクローンする
!mkdir -p .tmp
!git clone --depth 1 --filter=blob:none $input_repo /home/jovyan/.tmp/$input_repo_title

In [ ]:
import json
import glob 
from ipywidgets import Dropdown, Button, Layout
from IPython.display import clear_output


# 入力リポジトリの実験群を取得
input_repo_path = '/home/jovyan/.tmp/' + input_repo.split('/')[-1].replace('.git', '')
with open(input_repo_path + '/pipeline.json', 'r') as f:
    pipeline = json.load(f)

# 選択操作
style = {'description_width': 'initial'}
print('取得したいデータを持つ実験パッケージ名を以下から選択し、入力完了ボタンをクリック下さい。')
    
def on_click_package(clicked_button: Button) -> None:
    def on_click_parameter(clicked_button: Button) -> None:
        global parameter
        print("HERE")
        parameter=dropdown_parameter.value
        clear_output()
        print('入力を受けつけました。')
        print('実験パッケージ名：', package)
        print('パラメータ実験名：', parameter)
    
    global package
    package=dropdown_package.value
    clear_output()
    print('入力を受けつけました。')
    print('実験パッケージ名：', package)
    
    # RCOS_for_parametersの場合は、パラメータフォルダも選択
    with open(input_repo_path + '/dmp.json', 'r') as f:
        global dmp
        dmp = json.load(f)
    if dmp['datasetStructure'] == 'RCOS_for_parameters':
        # パラメータフォルダ一覧を生成
        output_dirs = glob.glob(input_repo_path + '/experiments/' + package + '/**/output_data/', recursive=True)
        parameter_dirs = [dir.replace('/output_data/', '') for dir in output_dirs]
        parameter_dirs = [dir.replace(input_repo_path + '/experiments/' + package + '/', '') for dir in parameter_dirs]
        clear_output()
        print('取得したい出力データを持つパラメータ実験名を選択し、入力完了ボタンをクリック下さい。')
        
        dropdown_parameter = Dropdown(
            options=parameter_dirs,
            description='パラメータ実験名:',
            disabled=False,
            layout=Layout(width='initial'),
            style=style
        )
        button_parameter = Button(description='入力完了する')
        button_parameter.on_click(on_click_parameter)
        display(dropdown_parameter, button_parameter)
    else:
        pass

dropdown_package = Dropdown(
    options=pipeline,
    description='実験パッケージ名:',
    disabled=False,
    layout=Layout(width='initial'),
    style=style
)

button_package = Button(description='入力完了')
button_package.on_click(on_click_package)
display(dropdown_package, button_package)

## 3. 必要なファイルを選択する

以下のセルを実行すると、手順2.で選択した実験にあるファイルが表示されます。  
表示されたファイルからこの実験に必要なファイルをクリックで選択してください。  
Ctrlキーを押しながらクリックすると複数のファイルを選択できます。

In [ ]:
import glob
import panel as pn
from pathlib import Path

pn.extension()
package_path = input_repo_path + '/experiments/' + package

def get_files(target, parameter):
    if parameter == '':
        cmd_glob = package_path + '/' + target + '/**'
        cmd_replace = package_path + '/' + target + '/'
    else:
        cmd_glob = package_path + '/' + parameter + '/' + target + '/**'
        cmd_replace = package_path + '/' + parameter + '/' + target + '/'
    files = glob.glob(cmd_glob, recursive=True)
    # フォルダを除く
    for i in files:
        p = Path(i)
        if Path.is_dir(p):
            files.remove(i)
    return [file.replace(cmd_replace, '') for file in files]

def generate_gui(files_list):
    global gui_list
    gui_list = []
    for key, value in files_list.items():
        if key == 'input_data' or  key == 'source' or key =='output_data':
            gui_list.append(pn.widgets.MultiSelect(name=key, options=value, size=8))
        elif key == 'parameter_output_data':
            gui_list.append(pn.widgets.MultiSelect(name=parameter + '/' + key.replace('parameter_',''), options=value, size=8))
    return gui_list
            
# 実験パッケージのタイプごとにGUIを表示する
if dmp['datasetStructure'] == 'RCOS_with_code':
    input_data_files = get_files(target='input_data', parameter='')
    source_files = get_files(target='source', parameter='')
    output_data_files = get_files(target='output_data', parameter='')
    files_list = {"input_data":input_data_files, "source":source_files, "output_data":output_data_files}
    gui = generate_gui(files_list)
    
elif dmp['datasetStructure'] == 'RCOS_only_data':
    input_data_files = get_files(target='input_data', parameter='')
    output_data_files = get_files(target='output_data', parameter='')
    files_list ={"input_data":input_data_files, "output_data":output_data_files}
    gui = generate_gui(files_list)
    
elif dmp['datasetStructure'] == 'RCOS_for_parameters':
    input_data_files = get_files(target='input_data', parameter='')
    source_files = get_files(target='source', parameter='')
    output_data_files = get_files(target='output_data', parameter=parameter)
    files_list = {"input_data":input_data_files, "source":source_files, "parameter_output_data":output_data_files}
    gui = generate_gui(files_list)
    
column = pn.Column()
for target in gui:
    column.append(target)
column

## 4. 選択したファイルの格納先を入力する

手順3で選択したファイルの格納先について、実験パッケージからファイル名までのパスを入力してください。
例えば、`input_data/experiment_1_output/sample.txt`などです。
<p style="color:red;">格納先のファイルパスは、`input_data/`, `source/`, `output_data/`で始まる必要があります。</p> 

In [ ]:
gui_list = []
for i in range(len(column)):
    if len(column[i].value) > 0:
        gui_list.append('### ' + column[i].name)
    for index in range(len(column[i].value)):
        gui_list.append(pn.widgets.TextInput(name=column[i].name + '/' + column[i].value[index], placeholder='Enter a file path here...'))

column = pn.Column()
for target in gui_list:
    column.append(target)
column

## 5. 選択されたファイルを指定された格納先に用意する

In [ ]:
import os
import csv
from datalad import api

dest = {}
%store -r EXPERIMENT_TITLE
# datalad addurlsで使うファイルのダウンロードリンクと格納先のcsvファイルを生成する
for i in range(len(column)):
    panel_type = str(type(column[i]))
    if 'TextInput' in panel_type:
        cmd = package_path + '/' + column[i].name
        annex_key = !git annex info $cmd | grep key
        if 'URL' in annex_key[0].split('-')[0]:
            # S3などインターネット上に実体がある場合
            pass
        else:
            # 実体がGIN-forkにある場合
            url = input_repo.replace('.git', '') + '/raw/master/experiments/' + package + '/' + column[i].name
            dest_path = 'experiments/' + EXPERIMENT_TITLE + '/' + column[i].value
            dest[url] = dest_path
with open('/home/jovyan/.tmp/datalad-addurls.csv', 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(['who','link'])
    for key, value in dest.items():
        writer.writerow([value, key])

In [ ]:
# addurlsを実行してgit annexリンクを作成する
# annex状態でいったんコミットしておく
import os
os.chdir(os.environ['HOME'])
result = ''
try:
    result = !datalad addurls --nosave --fast .tmp/datalad-addurls.csv '{link}' '{who}'
    for line in result:
        if 'addurl(error)' in line  or 'addurl(impossible)' in line:
            raise Exception
except Exception:
    print('リンク情報の作成に失敗しました。')
else:
    print('リンク情報の作成に成功しました。次の処理にお進みください。')
finally:
    print(result)

In [ ]:
from datalad import api

datalad_get_path = []
for value in dest.values():
    datalad_get_path.append(value)

# 来歴を残すためannex管理でコミットする
api.save(path=datalad_get_path, message='prepare_from_repository : addurls')

# 実データを取得する
api.get(path=datalad_get_path)

# sourceがあれば、git管理に戻す
source_path = []
for path in datalad_get_path:
    if path.startswith('experiments/' + EXPERIMENT_TITLE + '/source'):
        source_path.append(path)
if len(source_path) > 0:
    for path in source_path:
        !git annex unannex $path
        
# 一時的なクローンを削除する
os.chdir(os.environ['HOME'])
!rm -rf .tmp/$input_repo_title

## 6. 本タスクの実行結果をGIN-forkに同期する

ここまでの内容を保存し、GIN-forkに同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

save_path = source_path
annexed_save_path = list(set(datalad_get_path) - set(save_path))

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
import papermill as pm
from colorama import Fore
from IPython.display import clear_output

os.chdir(os.environ['HOME'])
save_path.append('WORKFLOWS/EX-WORKFLOWS/prepare_from_repository.ipynb')
# Git管理ファイルを保存
try:
    pm.execute_notebook(
        'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters = dict(SAVE_MESSAGE =EXPERIMENT_TITLE + '_実験データの準備 (1/2)', TO_GIT = True, PATH = save_path, IS_RECURSIVE = False)
    )
finally:
    clear_output()
    %store -r DATALAD_MESSAGE
    %store -r DATALAD_ERROR
    print('\n' + DATALAD_MESSAGE + '\n')
    print(Fore.RED + DATALAD_ERROR)

In [ ]:
import os
import papermill as pm
from colorama import Fore
from IPython.display import clear_output

os.chdir(os.environ['HOME'])
# Git-annex管理ファイルを保存
try:
    pm.execute_notebook(
        'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_実験データの準備 (2/2)', PATH = annexed_save_path, IS_RECURSIVE = False, UNLOCK = False, REMOVE_PUSH_PATH = True)
    )
finally:
    clear_output()
    %store -r DATALAD_MESSAGE
    %store -r DATALAD_ERROR
    print('\n' + DATALAD_MESSAGE + '\n')
    print(Fore.RED + DATALAD_ERROR)

## 7. 実験ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../experiment.ipynb)からトップページに遷移できます。  